References:
* https://colab.ampl.com/index.html
* https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/marcos-dv/puzzles/sudoku_gen.ipynb

A partially filled Sudoku is called "irreducible" if there is only one way to complete it, and removing any of the clues allows multiple solutions.

Find out whether a Sudoku is irreducible or not is not straightforward. We are integrating AMPL and Highs within an iterative method to generate irreducible puzzles.

In [1]:
%pip install -q amplpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.0 MB/s eta 0:00:00


In [2]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://colab.ampl.com).


In [31]:
%%ampl_eval
reset;

# PARAMETERS AND SETS
param n; # Sudoku size, usually n=3
set ROWS := 1..n^2;
set COLS := 1..n^2;
set DIGITS := 1..n^2;
set SUBSQUARES{sr in 1..n, sc in 1..n} =
    setof {i in ROWS, j in COLS : sr = ceil(i/n) and sc = ceil(j/n)} (i,j);

param T{ROWS, COLS} default 0 <= n^2; # Template
param Weights{ROWS, COLS, DIGITS} default 0; # Objective function

# VARIABLES
var x{ROWS,COLS,DIGITS}, binary;

# CONSTRAINTS
Diff_Rows{i in ROWS, k in DIGITS}:
	sum{j in COLS} x[i,j,k] = 1;

Diff_Cols{j in COLS, k in DIGITS}:
	sum{i in ROWS} x[i,j,k] = 1;

Diff_Subsq{sr in 1..n, sc in 1..n, k in DIGITS}:
	sum{(i,j) in SUBSQUARES[sr,sc]} x[i,j,k] = 1;

One_Digit{i in ROWS, j in COLS}:
	sum{k in DIGITS} x[i,j,k] = 1;

Fill_Tile{i in ROWS, j in COLS : T[i,j] > 0}:
	x[i,j,T[i,j]] = 1;

minimize f: sum{i in ROWS, j in COLS : T[i,j] > 0} Weights[i,j,T[i,j]]*x[i,j,T[i,j]];

In [32]:
import itertools

n = 3
ampl.param["n"] = n

solution = [
    [1, 2, 3, 4, 5, 6, 7, 8, 9],
    [4, 5, 6, 7, 8, 9, 1, 2, 3],
    [7, 8, 9, 1, 2, 3, 4, 5, 6],
    [2, 3, 1, 5, 6, 4, 8, 9, 7],
    [5, 6, 4, 8, 9, 7, 2, 3, 1],
    [8, 9, 7, 2, 3, 1, 5, 6, 4],
    [3, 1, 2, 6, 4, 5, 9, 7, 8],
    [6, 4, 5, 9, 7, 8, 3, 1, 2],
    [9, 7, 8, 3, 1, 2, 6, 4, 5],
]

rows = range(1, n * n + 1)  # 1..n^2
ampl.param["T"] = {
    (i, j): solution[i - 1][j - 1] for i, j in itertools.product(rows, rows)
}
ampl.option["solver"] = "highs"
ampl.solve()

Solution determined by presolve;
objective f = 0.


In [33]:
import random
import time

# Step 2
x = ampl.var["x"]
solution = {
    (i, j): k
    for (i, j, k) in itertools.product(rows, rows, rows)
    if x[i, j, k].value() > 0
}
# Initial template
ampl.param["T"] = solution
tiles = list(itertools.product(rows, rows))
# Tiles are shuffled, as the very first tiles are going to be cleared.
random.shuffle(tiles)

In [34]:
t0 = time.time()
# Step 3
for i, j in tiles:
    # Step 3.1
    # Clear clue + Add variable to the objective
    ampl.eval("drop Fill_Tile[" + str(i) + "," + str(j) + "];")
    ampl.param["Weights"][
        i, j, solution[i, j]
    ] = 1  # minimize x[i,j,T[i,j]] is the objective
    # Solve
    ampl.solve()
    # Step 3.2
    # Check if the objective changed
    if ampl.obj["f"].value() > 0:  # Unique solution: Clear clue
        ampl.param["T"][i, j] = 0
    else:  # Multiple solution: Fix tile
        ampl.eval("restore Fill_Tile[" + str(i) + "," + str(j) + "];")
    # Take out from the objective
    ampl.param["Weights"][i, j, solution[i, j]] = 0
t1 = time.time()

Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.
Solution determined by presolve;
objective f = 1.


In [35]:
def show_sudoku(T):
    for i in rows:
        for j in rows:
            print(
                int(T[i, j]) if int(T[i, j]) > 0 else ".",
                end=" | " if j % n == 0 and j < n * n else "",
            )
        print("\n" + "-" * ((n - 1) * 3 + n * n) if i % n == 0 else "")


print("--- Sudoku template with unique solution:\n")
show_sudoku(ampl.param["T"])
print("Time: " + str(t1 - t0) + " seconds")

--- Sudoku template with unique solution:

..3 | .5. | ...
.5. | ..9 | 12.
7.. | 1.. | ...
---------------
.3. | ... | ..7
..4 | ... | 2..
.9. | 231 | 5..
---------------
.1. | 6.5 | 9..
... | 9.8 | ..2
... | ... | .4.
---------------
Time: 1.008357286453247 seconds
